In [85]:
import pandas as pd
import os
import numpy as np

In [3]:
os.chdir('/Users/ianjeong/Desktop/ML/great_cappsby/crypto-currency/data')

In [86]:
bitcoin = pd.read_csv('bitcoin_daily.csv')

In [88]:
# find daily change in closing price
bitcoin['Daily_Change'] = bitcoin['Close'] - bitcoin['Open']

In [96]:
def expo_moving_average(df, period):
    '''
    Calculate the exponential moving average.
    
    Input:
        df: a pandas dataframe
        period (int): time period
    Returns:
        rv: a pd series of exponential moving averages
    '''
    col_name = f'EMA_{period}' 
    rv = pd.Series([None] * df.shape[0])
    weight_0 = 2 / (period + 1) 
    weight_1 = 1 - weight_0
    for row in df.itertuples():
        if row.Index < period - 1:
            continue
        elif row.Index == period - 1:
            simple_average = df.loc[:period - 1, 'Close'].sum() / period
            #df.loc[row.Index, col_name] = simple_average
            rv[row.Index] = simple_average
        else:
            #ema = row.Close * weight_0 + df.loc[row.Index - 1, col_name] * weight_1
            ema = row.Close * weight_0 + rv[row.Index - 1] * weight_1
            #df.loc[row.Index, col_name] = ema
            rv[row.Index] = ema
    return rv

In [94]:
expo_moving_average(bitcoin, 12)
expo_moving_average(bitcoin, 26)

0               None
1               None
2               None
3               None
4               None
            ...     
2742    40427.569083
2743    40570.623861
2744     40742.64854
2745    40981.040026
2746     41233.99742
Length: 2747, dtype: object

In [97]:
def moving_average_convergence_divergence(df):
    '''
    Find the moving average convergence divergence, which is equal to the
    difference between the 12-period exponential moving average and
    26-period exponential moving average.
    
    Modifies the dataframe in place.
    
    Input:
        df: a pd dataframe
    '''
    col_name = 'MACD'
    ema_12 = expo_moving_average(bitcoin, 12)
    ema_26 = expo_moving_average(bitcoin, 26)
    df['MACD'] = ema_12 - ema_26

In [53]:
# find the moving average converage divergence
bitcoin['MACD'] = bitcoin['EMA_12'] - bitcoin['EMA_26']

In [98]:
moving_average_convergence_divergence(bitcoin)

In [99]:
def proc(df, period):
    '''
    Calculate the price rate of change.
    
    Input:
        df: a pandas dataframe
        period (int): time period
    '''
    
    col_name = f'PROC_{period}'
    df[col_name] = None
    for row in df.itertuples():
        if row.Index < period:
            continue
        prev_idx = row.Index - period
        df.loc[row.Index, col_name] = (df.loc[row.Index, 'Close'] - df.loc[prev_idx, 'Close']) / df.loc[prev_idx, 'Close']

In [100]:
proc(bitcoin, 3)
proc(bitcoin, 5)
proc(bitcoin, 10)

In [103]:
def williams_percentage_range(df):
    '''
    Find the Williams Percentage Range. Modifies the df in place. 
    
    Input:
        df: a pd dataframe
    '''
    col_name = 'wpr'
    df[col_name] = None
    for row in df.itertuples():
        if row.Index < 14:
            continue
        max_14 = df.loc[row.Index - 14:row.Index - 1, 'Close'].max()
        min_14 = df.loc[row.Index - 14:row.Index - 1, 'Close'].min()
        wpr = (max_14 - row.Close) / (max_14 - min_14) * -100
        df.loc[row.Index, col_name] = wpr

In [104]:
williams_percentage_range(bitcoin)

In [105]:
def stochastic_oscillator(df):
    '''
    Find the stochastic oscillator. Modifies the df in place.
    '''
    col_name = 'sto_os'
    df[col_name] = None
    for row in df.itertuples():
        if row.Index < 14:
            continue
        max_14 = df.loc[row.Index - 14:row.Index - 1, 'Close'].max()
        min_14 = df.loc[row.Index - 14:row.Index - 1, 'Close'].min()
        so = (row.Close - min_14) / (max_14 - min_14) * 100
        df.loc[row.Index, col_name] = so

In [106]:
stochastic_oscillator(bitcoin)

In [107]:
bitcoin.head(30)

,Date,Open,High,Low,Close,Adj Close,Volume,Daily_Change,MACD,PROC_3,PROC_5,PROC_10,wpr,sto_os
0,2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800,-8.529999,NaN,None,None,None,None,None
1,2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200,-32.419983,NaN,None,None,None,None,None
2,2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700,-29.307007,NaN,None,None,None,None,None
3,2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600,14.230988,NaN,-0.105896,None,None,None,None
4,2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100,-9.263977,NaN,-0.06036,None,None,None,None
5,2014-09-22,399.100006,406.915985,397.130005,402.152008,402.152008,24127600,3.052002,NaN,0.018632,-0.12066,None,None,None
6,2014-09-23,402.092010,441.557007,396.196991,435.790985,435.790985,45099500,33.698975,NaN,0.065754,0.026743,None,None,None
7,2014-09-24,435.751007,436.112000,421.131989,423.204987,423.204987,30627700,-12.546020,NaN,0.06114,0.071959,None,None,None
8,2014-09-25,423.156006,423.519989,409.467987,411.574005,411.574005,26814400,-11.582001,NaN,0.023429,0.00653,None,None,None
9,2014-09-26,411.428986,414.937988,400.009003,404.424988,404.424988,21460800,-7.003998,NaN,-0.071975,0.014051,None,None,None


In [108]:
bitcoin.to_csv('bitcoin_daily_modified.csv')